<a href="https://colab.research.google.com/github/pinak139/Temp/blob/main/Copy_of_Glassnost_V_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pulp as plp
import pandas as pd
import polars as plr
from docplex.mp.model import Model
from pulp import *
from matplotlib import pyplot as plt
# Get Solver
solver_list = plp.listSolvers(onlyAvailable=True)
solver = plp.getSolver('CPLEX_CMD')

In [ ]:
product_list = [5478727]

In [ ]:
location = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'LOCATION')
product = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'PRODUCT')
product = product[product['ProductID'].isin(product_list)]
products = list(product['ProductID'].unique())
locationProduct = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'LOCATIONPRODUCT')
locationProduct = locationProduct[locationProduct['ProductID'].isin(products)]
supplier = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'SUPPLIER')
supplierProduct = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'SUPPLIERPRODUCT')
supplierProduct = supplierProduct[supplierProduct['pProductID'].isin(products)]
openpo = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'OPENPO')
openpo = openpo[openpo['ixSKU'].isin(products)]
opensto = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'OPENSTO')
opensto = opensto[opensto['ixSKU'].isin(products)]
opensto = opensto[['ixVendor','ixSKU','nodeID','ixFromLocation','ixToLocation','iQuantityRequested']].groupby(['ixVendor','ixSKU','nodeID']).max().reset_index()
currentinv = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'CURRENTINVENTORY')
currentinv = currentinv[currentinv['ixSKU'].isin(products)]

forecast = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'FORECAST')
forecast = forecast[forecast['ixSKU'].isin(products)]

calendar = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'CALENDAR')
calmondays = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'CALENDAR_MONTH_DAYS')
alloc = pd.read_excel('I:\Consulting\Julio_Data_Template_V3_0.xlsx',sheet_name = 'ALLOC')
alloc = alloc[alloc['ixSKU'].isin(products)]

temp = calendar[['MONTHSTARTDATE','FISCAL_DATE']].merge(forecast, left_on = 'MONTHSTARTDATE',right_on = 'ForecastedPeriod', how = 'inner')
temp = temp.merge(calmondays, on = 'MONTHSTARTDATE', how = 'inner')
temp['DailyForecast'] = temp['Forecast']/temp['NBR_DAYS_IN_MONTH']
forecast = temp[['FISCAL_DATE','ixSKU','DailyForecast']]

locationProduct['nodeID'] = list(zip(locationProduct.LocationID, locationProduct.ProductID))
currentinv['nodeID'] = list(zip(currentinv.ixLocation, currentinv.ixSKU))
opensto['nodeID'] = list(zip(opensto.ixToLocation, opensto.ixSKU))
internal_node = locationProduct[['LocationID','ProductID','nodeID','pPrimarySupplierID','nLeadTime','nWACC','nInvTgtDays','nInvMinDays','nLeadTime']].merge(currentinv[['nodeID','ixSKU','nInitInv']], on = 'nodeID', how = 'left')

central_node = internal_node[internal_node['LocationID'] == 99]
sat_locs = [25,85]
sat_node =  internal_node[internal_node['LocationID'].isin(sat_locs)]

opensto = opensto.drop_duplicates()


central_node = central_node.merge(openpo, on = 'ixSKU', how = 'left').fillna(0)
#sat_node = internal_node[internal_node['LocationID'].isin(sat_locs)]
sat_node = sat_node.merge(opensto, on = 'nodeID',how = 'left')
sat_node['ProductID'] = sat_node['ixSKU_x']
sat_node = sat_node[['LocationID','ProductID','nodeID','nLeadTime','nInvTgtDays','nInvMinDays','nInitInv','iQuantityRequested']]
sat_node = sat_node.fillna(0)


In [ ]:
# Data Prep




alloc = alloc[['ixSKU','Location','DailyDemand']].merge(location[['LocShort','LocationID']], left_on = 'Location', right_on = 'LocShort')

node_forecast = forecast.merge(alloc, on = 'ixSKU', how = 'inner')
node_forecast['NodeDailyForecast'] = node_forecast['DailyForecast']* node_forecast['DailyDemand']
node_forecast = node_forecast[['FISCAL_DATE','ixSKU','LocationID','NodeDailyForecast']]

product['ProductID'] = product['ProductID'].astype(str)
locationProduct['ProductID'] = locationProduct['ProductID'].astype(str)
supplierProduct['pProductID'] = supplierProduct['pProductID'].astype(str)
openpo['ixSKU'] = openpo['ixSKU'].astype(str)
opensto['ixSKU'] = opensto['ixSKU'].astype(str)
currentinv['ixSKU'] = opensto['ixSKU'].astype(str)
forecast['ixSKU'] = forecast['ixSKU'].astype(str)
alloc['ixSKU'] = alloc['ixSKU'].astype(str)



# Internal Nodes

central_node['InvOO'] = central_node['OpenPOQty']
#central_node[''] = central_node['OpenPOQty']
central_node = central_node[['nodeID','LocationID','ProductID','pPrimarySupplierID','nLeadTime','nWACC','nInvTgtDays','nInvMinDays','nInitInv','InvOO']]
central_node['nodeType'] = 'central'



sat_node['pPrimarySupplierID'] = 99
#sat_node['ixSKU'] = sat_node['ixSKU_x']
sat_node['InvOO'] = sat_node['iQuantityRequested']
sat_node['nWACC'] = 0.08

sat_node = sat_node[['nodeID','LocationID','ProductID','pPrimarySupplierID','nLeadTime','nWACC','nInvTgtDays','nInvMinDays','nInitInv','InvOO']]
sat_node['nodeType'] = 'satellite'

node = pd.concat([central_node,sat_node])


#############


#Periods

periods = list(node_forecast['FISCAL_DATE'].unique())
numPeriods = range(1,len(periods) + 1)
period_dict =  {'periodID': numPeriods, 'FISCAL_DATE': periods}
period_df = pd.DataFrame(period_dict)

# Forecast

from datetime import datetime,timedelta
def first_day_of_week(date):
    return date - timedelta(days = date.weekday())

period_df['WeekStart'] = period_df['FISCAL_DATE'].apply(first_day_of_week)
numWks = len(list(period_df['WeekStart'].unique()))
Weeks = list(range(1,numWks + 1))

node_forecast = node_forecast.merge(period_df, on  = 'FISCAL_DATE')
node_forecast = node_forecast.groupby(['WeekStart','ixSKU','LocationID']).sum('NodeDailyForecast').reset_index()

timep = list(period_df['WeekStart'].unique())
periods = pd.DataFrame({'WeekStart': timep,'weekID': Weeks})

weekforecast = node_forecast.merge(periods, on = 'WeekStart')
weekforecast['forecastID'] = list(zip(weekforecast.LocationID,weekforecast.ixSKU,weekforecast.weekID))



# Forecast Prep

weekforecast = weekforecast.set_index('forecastID')
weekforecast['NodeDailyForecast'] = weekforecast['NodeDailyForecast']
weekforecast['nodeID'] = list(zip(weekforecast.LocationID, weekforecast.ixSKU))
dWeekOut = weekforecast['weekID'].to_dict()
dWeekStart = weekforecast['WeekStart'].to_dict()
dProduct = weekforecast['ixSKU'].to_dict()
dLocation = weekforecast['LocationID'].to_dict()
dForecast = weekforecast['NodeDailyForecast'].to_dict()
dDayOut = weekforecast['periodID'].to_dict()
dNode = weekforecast['nodeID'].to_dict()

# Periods

periods = list(weekforecast['weekID'].unique())
#periods = periods[:9]


# Location MAster

location = location.set_index('LocationID')
lLocShort = location['LocShort'].to_dict()
lCity = location['City'].to_dict()
lState = location['State'].to_dict()



# Product Master

product = product.set_index('ProductID')
pName = product['pProductName'].to_dict()
pPrice = product['pPrice'].to_dict()
pVolume = product['pVolume'].to_dict()
pMOQ = product['pMOQ'].to_dict()
pCost = product['pCost'].to_dict()
pLeadTime = product['pLeadTime'].to_dict()
pPrimarySupplierID = product['pPrimarySupplierID'].to_dict()
pPrimarySupplierName = product['pPrimarySupplierName'].to_dict()


# Vendor Master

supplier = supplier.set_index('sSupplierID')
sVendorName = supplier['sSupplierName'].to_dict()
sCity = supplier['sSupplierCity'].to_dict()
sState = supplier['sSupplierState'].to_dict()
sPaymentTerms = supplier['sPaymentTerms'].to_dict()
sFreightTerms = supplier['sFreightTerms'].to_dict()
sOverseas = supplier['flgOverseas'].to_dict()
sMinPODollars = supplier['mMinPODollars'].to_dict()
sMinFreightDollars = supplier['mMinFreightDollars'].to_dict()


# Vendor Nodes


supplierProduct['supplierProductID'] = list(zip(supplierProduct.pVendorID,supplierProduct.pProductID))
supplierProduct =  supplierProduct.set_index('supplierProductID')
spCost = supplierProduct['mCost'].to_dict()



sat_node = sat_node.set_index('nodeID')
central_node = central_node.set_index('nodeID')
node = node.set_index('nodeID')
nLocation = node['LocationID'].to_dict()
nProduct = node['ProductID'].to_dict()
nVendor = node['pPrimarySupplierID'].to_dict()
nLeadTime = node['nLeadTime'].to_dict()
nWACC = node['nWACC'].to_dict()
nInvTgtDays = node['nInvTgtDays'].to_dict()
nInvMinDays = node['nInvMinDays'].to_dict()
nInvOO = node['InvOO'].to_dict()
nType = node['nodeType'].to_dict()
nInitInv = node['nInitInv'].to_dict()

# Index Sets

locations  = list(location.index)
products = list(product.index)

nodes = list(node.index)
sat_nodes = list(node[node['nodeType'] == 'satellite'].index)
central_nodes = list(node[node['nodeType'] == 'central'].index)
suppliers = list(supplier.index)


# ForecastID
forecasts = list(weekforecast.index)

node = node.reset_index()

lane = node[['LocationID','pPrimarySupplierID','ProductID','nLeadTime','InvOO']]
lane = lane.rename(columns = {'LocationID': 'LocTo', 'pPrimarySupplierID':'LocFrom','nLeadTime': 'lnLeadTime'})
lane['LaneID'] = list(zip(lane.LocFrom, lane.LocTo,lane.ProductID))
lane['NodeFrom'] = list(zip(lane.LocFrom,lane.ProductID))
lane['NodeTo'] = list(zip(lane.LocTo, lane.ProductID))
lane = lane.set_index('LaneID')

lnLocFrom = lane['LocFrom'].to_dict()
lnLocTo = lane['LocTo'].to_dict()
lnProduct = lane['ProductID'].to_dict()
lnNodeFrom = lane['NodeFrom'].to_dict()
lnNodeTo = lane['NodeTo'].to_dict()
lnLeadTime = lane['lnLeadTime'].to_dict()
lInitInTransit = lane['InvOO'].to_dict()

lanes = list(lane.index)

In [ ]:
# Decision Variables

from pulp import *

# Decision Variables

model = LpProblem('JULIO_V_1_1: General Sourcing andn Distribution Model- Last Mile', LpMinimize)

# CORE VARIABLES

INV = LpVariable.dicts("InventoryOnHand", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
FCST = LpVariable.dicts("Forecast", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
TSREQ = LpVariable.dicts("TransferReq", [(l,p,t) for l,p,t in forecasts ] , lowBound = 0, cat = 'Continuous')
TSREQm = LpVariable.dicts("TransferReq", [(l,p,t) for l,p,t in forecasts] , lowBound = 0, cat = 'Continuous')
TSREC = LpVariable.dicts("TransferRec", [(l,p,t) for l,p,t in forecasts] , lowBound = 0, cat = 'Continuous')
DEPREQ = LpVariable.dicts("DependentReq", [(l,p,t) for l,p,t in forecasts] , lowBound = 0, cat = 'Continuous')
PREQ = LpVariable.dicts("PurchaseReq", [(l,p,t)  for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
PREQm = LpVariable.dicts("PurchaseReqSlack", [(l,p,t)  for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
PREC = LpVariable.dicts("PurchaseRec", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
STOO = LpVariable.dicts("StockTransferOnOrder", [(l,p,t) for  l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
POO = LpVariable.dicts("PurchaseOnOrder", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
DEL = LpVariable.dicts("Delivered", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
DIS = LpVariable.dicts("Delivered", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
# Calc Variables
BAS = LpVariable.dicts("BaseStk", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
SAF = LpVariable.dicts("SafStk", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
OUL = LpVariable.dicts("SafStk", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
# Slack Variables

MIS = LpVariable.dicts("MissedSales", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
INVm = LpVariable.dicts("InventoryMinSlack", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
INVp = LpVariable.dicts("InventoryMaxSlack", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
BUYm = LpVariable.dicts("PurchaseMinSlack", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')
BUYp = LpVariable.dicts("PurchaseMaxSlack", [(l,p,t) for l,p,t in forecasts], lowBound = 0, cat = 'Continuous')

AGGm = LpVariable.dicts("MinVendorSlack", [(s,t) for s in nodes for t in periods], lowBound = 0, cat = 'Continuous')
AGGm = LpVariable.dicts("MinVendorSlack", [(s,t) for s in nodes for t in periods], lowBound = 0, cat = 'Continuous')

In [ ]:
# Constraints

nbPeriods = 52

#dForecast

for l,p,t in forecasts:
    if t < nbPeriods:
        model += FCST[l,p,t] == dForecast[l,p,t]


for l,p,t in forecasts:
    if t < nbPeriods:
        model += FCST[l,p,t] == DIS[l,p,t] + MIS[l,p,t]

for l,p,t in forecasts:
    if t < nbPeriods:
        if nType[l,p] == 'satellite':
            if t == 1:
                model += INV[l,p,t] == nInitInv[l,p] + TSREC[l,p,t] - DIS[l,p,t]
            if t > 1:
                model += INV[l,p,t] == INV[l,p,t-1] + TSREC[l,p,t] - DIS[l,p,t]




for l,p,t in forecasts:
    if t < nbPeriods:
        if nType[l,p] == 'satellite':
            model += INV[l,p,t] >= FCST[l,p,t]*5 - INVm[l,p,t]



for l,p,t in forecasts:
    if t < nbPeriods:
        if nType[l,p] == 'satellite':
            model += INV[l,p,t] <= FCST[l,p,t]*nInvTgtDays[l,p] + INVp[l,p,t]





for l,p,t in forecasts:
    if t < nbPeriods and t > 4:
        if nType[l,p] == 'satellite':
            model += TSREQ[l,p,t] == TSREC[l,p,t]
"""
for l,p,t in forecasts:
    if t < nbPeriods and t > 4:
        if nType[l,p] == 'central':
            model += DEPREQ[l,p,t] == lpSum(TSREQ[lTo,p,t] for lTo in locations if nVendor[lTo,p] == l)
"""

In [ ]:
# Capturing Outputs

location = []
product = []
period = []
inventory = []
inventorycost = []
forecast = []
delivered = []
missed = []

transferRec = []
transferReq = []
for l,p,t in forecasts:
    if t < nbPeriods:
        if nType[l,p] == 'satellite':
            location.append(l)
            product.append(p)
            period.append(t)
            inventory.append(INV[l,p,t].varValue)
            forecast.append(FCST[l,p,t].varValue)
            delivered.append(DEL[l,p,t].varValue)
            missed.append(MIS[l,p,t].varValue)
            transferRec.append(TSREC[l,p,t].varValue)
            transferReq.append(TSREQ[l,p,t].varValue)


sat_stats = pd.DataFrame(zip(location,product,period,inventory,forecast,transferRec,transferReq),columns = ['LocationID','ProductID','Period','Inventory','Forecast','TransferReceipts','TransferReq'])

#print (sat_stats)

In [ ]:
location = []
product = []
period = []
inventory = []
forecast = []
depreq = []


for l,p,t in forecasts:
    if nType[l,p] == 'central':
        location.append(l)
        product.append(p)
        period.append(t)
        inventory.append(INV[l,p,t].varValue)
        forecast.append(FCST[l,p,t].varValue)
        depreq.append(DEPREQ[l,p,t].varValue)

central_stats = pd.DataFrame(zip(location,product,period,inventory,forecast,depreq),
                columns = ['LocationID','ProductID','Period','InventoryOnHand','Forecast','DepReq'])

central_stats['Demand'] = central_stats['Forecast'] + central_stats['DepReq'].fillna(0)